__Keras Deep Learning Model:__

Doesn't work :D 25% accuracy. 

In [69]:
import keras
import pickle
from keras.layers import Input, Dense, Add, Concatenate
from keras.models import Model
from keras.utils import plot_model
from sklearn.model_selection import train_test_split

In [14]:
with open('FeatureData/featuresnew.pk', 'rb') as file:
    warren_features, warren_feature_names, warren_labels = pickle.load(file)

In [67]:
warren_feature_names[:8], len(warren_features), len(warren_features[0])

(['sadness',
  'joy',
  'fear',
  'disgust',
  'anger',
  'sentiment',
  'character',
  '#1job'],
 1742,
 5719)

In [70]:
warren_features_train, warren_features_test, y_train, y_test = train_test_split(warren_features, warren_labels, test_size=1/3, random_state=42)

In [71]:
warren_emot_mat = [i[:5] for i in warren_features_train]
warren_sent_mat = [i[5] for i in warren_features_train]
warren_char_mat = [i[6] for i in warren_features_train]
warren_keys_mat = [i[7:] for i in warren_features_train]
# [warren_features[:5], warren_features[5], warren_features[6], warren_features[7:]]
warren_emot_mat_test= [i[:5] for i in warren_features_test]
warren_sent_mat_test = [i[5] for i in warren_features_test]
warren_char_mat_test = [i[6] for i in warren_features_test]
warren_keys_mat_test= [i[7:] for i in warren_features_test]

In [76]:
len(warren_emot_mat), len(warren_emot_mat_test), len(warren_emot_mat[0]), len(warren_emot_mat_test[0])

(1161, 581, 5, 5)

In [77]:
def recall_m(y_true, y_pred):
        true_positives = keras.backend.sum(keras.backend.round(keras.backend.clip(y_true * y_pred, 0, 1)))
        possible_positives = keras.backend.sum(keras.backend.round(keras.backend.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + keras.backend.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = keras.backend.sum(keras.backend.round(keras.backend.clip(y_true * y_pred, 0, 1)))
        predicted_positives = keras.backend.sum(keras.backend.round(keras.backend.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + keras.backend.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+keras.backend.epsilon()))

In [86]:
emot1 = Input(shape=(5,),name='emot1')
emot2 = Dense(2, activation="sigmoid")(emot1)

sent1 = Input(shape=(1,),name='sent1')

char1 = Input(shape=(1,),name='char1')

keys1 = Input(shape=(5712,),name='keys1')
keys2 = Dense(60, activation='relu')(keys1)

merged = Concatenate()([emot2, sent1, char1, keys2])
merge_out = Dense(8)(merged)

out = Dense(1)(merge_out)

model = Model(inputs=[emot1, sent1, char1, keys1], outputs=out)


In [87]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
emot1 (InputLayer)              (None, 5)            0                                            
__________________________________________________________________________________________________
keys1 (InputLayer)              (None, 5712)         0                                            
__________________________________________________________________________________________________
dense_33 (Dense)                (None, 2)            12          emot1[0][0]                      
__________________________________________________________________________________________________
sent1 (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
char1 (Inp

In [88]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', f1_m])

In [89]:
len(warren_features[0])

5719

In [90]:
hist = model.fit([warren_emot_mat, warren_sent_mat, warren_char_mat, warren_keys_mat],
                 y_train,
                 epochs=100,
                 batch_size=16,
                 validation_data=([warren_emot_mat_test, warren_sent_mat_test,
                                  warren_char_mat_test, warren_keys_mat_test],y_test))

Train on 1161 samples, validate on 581 samples
Epoch 1/100
1161/1161 [==============================] - 1s 1ms/step - loss: -0.0348 - acc: 0.0000e+00 - f1_m: 0.0000e+00 - val_loss: -0.4909 - val_acc: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 2/100
1161/1161 [==============================] - 0s 420us/step - loss: -0.7181 - acc: 0.0000e+00 - f1_m: 0.0000e+00 - val_loss: -0.9383 - val_acc: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 3/100
1161/1161 [==============================] - 1s 454us/step - loss: -1.1826 - acc: 0.0078 - f1_m: 0.0363 - val_loss: -0.6240 - val_acc: 0.0069 - val_f1_m: 0.0349
Epoch 4/100
1161/1161 [==============================] - 1s 433us/step - loss: -1.4109 - acc: 0.0112 - f1_m: 0.0439 - val_loss: -0.5155 - val_acc: 0.0241 - val_f1_m: 0.1151
Epoch 5/100
1161/1161 [==============================] - 1s 440us/step - loss: -1.7019 - acc: 0.0189 - f1_m: 0.0745 - val_loss: -1.0236 - val_acc: 0.0069 - val_f1_m: 0.0349
Epoch 6/100
1161/1161 [==============================] - 1

Epoch 48/100
1161/1161 [==============================] - 0s 403us/step - loss: -5.9193 - acc: 0.5495 - f1_m: 1.6947 - val_loss: -1.5308 - val_acc: 0.2978 - val_f1_m: 0.9844
Epoch 49/100
1161/1161 [==============================] - 1s 433us/step - loss: -5.9205 - acc: 0.5426 - f1_m: 1.7583 - val_loss: -1.5425 - val_acc: 0.2995 - val_f1_m: 0.9844
Epoch 50/100
1161/1161 [==============================] - 0s 403us/step - loss: -5.9212 - acc: 0.5426 - f1_m: 1.6938 - val_loss: -1.5257 - val_acc: 0.3012 - val_f1_m: 0.9846
Epoch 51/100
1161/1161 [==============================] - 0s 390us/step - loss: -5.9219 - acc: 0.5383 - f1_m: 1.6847 - val_loss: -1.5066 - val_acc: 0.2995 - val_f1_m: 0.9739
Epoch 52/100
1161/1161 [==============================] - 0s 384us/step - loss: -5.9224 - acc: 0.5323 - f1_m: 1.6451 - val_loss: -1.4904 - val_acc: 0.2995 - val_f1_m: 0.9706
Epoch 53/100
1161/1161 [==============================] - 0s 387us/step - loss: -5.9229 - acc: 0.5297 - f1_m: 1.7136 - val_loss: -

In [84]:
model.weights[0]

<tf.Variable 'dense_29/kernel:0' shape=(5, 2) dtype=float32_ref>

In [13]:
hist.history

{'acc': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'f1_m': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.